## This notebook shows how to use Orchestrator APIs for Jupyterhub user experiments

In [6]:
import os
from fabric.cli.credential import CredMgr
from fabric.cli.orchestrator import Orchestrator

### Use $CILOGON_REFRESH_TOKEN to get FABRIC Tokens 

When user logins to Jupyterhub and authenticates herself against CILogon, OIDC refresh token is derived. <br>
Use this $CILOGON_REFRESH_TOKEN as parameter to get the FABRIC tokens.<br>
The FABRIC Identity Token is the token to use when communicating with Control Framework/Measurement Framework. <br>

In [7]:
refresh_res = CredMgr.refresh_token(project_name='all', scope='all', refresh_token=os.environ['CILOGON_REFRESH_TOKEN'])
print(refresh_res)
fabric_id_token=refresh_res['id_token']
fabric_refresh_token=refresh_res['refresh_token']
%env CILOGON_REFRESH_TOKEN=refresh_res['refresh_token']

Exception: ('INTERNAL SERVER ERROR', '"(\\"server_error\\"\\nerror_description=\\"Error: a null identifier was supplied\\"\\n) "\n')

In [ ]:
%store fabric_refresh_token

In [ ]:
print(os.environ['CILOGON_REFRESH_TOKEN'])

### Use Orchestrator API to query for available resources

In [ ]:
resources = Orchestrator.resources(id_token=fabric_id_token)

print(resources)